## Comparison of the results for the v2 VGB data

In [ ]:
### importing modules

import numpy as np
import os, sys, re
import time
import glob
import copy

from LISAhdf5 import LISAhdf5,ParsUnits
import tdi

import yaml, pandas, csv

import FastGB as FB

import LISAConstants as LC
import FrequencyArray


from tqdm import tqdm as tqdm
import corner

# from MCMC_multichain import MCcube  as mcmc

# import utils

from collections import defaultdict

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.style.use(['seaborn-ticks','seaborn-deep'])

# %pylab inline

mpl.rcParams.update({'font.size': 16})
plt.rcParams['axes.grid'] = True
plt.rcParams["figure.figsize"] = (12,7)


%load_ext autoreload
%autoreload 2

In [ ]:
### useful functions and parameters 

### Some settings
plotpal = ["#4C72B0", "#C44E52", "#CCB974", "#55A868", "#8172B2", "#64B5CD"]


def GetParams(p, ind):
    bet = p.get('EclipticLatitude')[ind]
    lam = p.get('EclipticLongitude')[ind]
    Amp = p.get("Amplitude")[ind]
    f0 = p.get("Frequency")[ind]
    fdot = p.get("FrequencyDerivative")[ind]
    iota = p.get("Inclination")[ind]
    psi = p.get("Polarization")[ind]
    phi0 = p.get("InitialPhase")[ind]

    return (bet, lam, Amp, f0, fdot, iota, psi, phi0)



def MakePrior1Src(f_fctr=1.e-3):
    Amp_bnd = [-24.0, -20.0] ### log10 amplitude
    #fr_bnd = [1.e-3, 7.e-3]
    # fr_bnd = np.array([1.0, 2.0])*f_fctr   ### in Hz
#     fr_bnd = np.array([10.06, 10.1])*f_fctr   ### in Hz
    fr_bnd = np.array([fr_my_min, fr_my_max])
    fdot_bnd = [-20.0, -14.0]  ### log10 fdot
    sin_bet_bnd = [-1.0, 1.0]
    lam_bnd = [0.0, 2.0*np.pi]
#     lam_bnd = [-np.pi, np.pi]
    cos_iota_bnd = [-1.0, 1.0]
    psi_bnd = [0.0, 2.0*np.pi]
    phi0_bnd = [0.0, 2.0*np.pi]

    prior = [Amp_bnd, fr_bnd, fdot_bnd, sin_bet_bnd, lam_bnd, cos_iota_bnd, psi_bnd, phi0_bnd]
    prior = np.array(prior)
    return (prior)


def Likelihood(Af, Ef):
    ib = Af.kmin
    ie = Af.kmin+len(Af) 
    fr = np.arange(Af.kmin, Af.kmin+len(Af))*df
    ### TODO I assume that the frequency range is the same for A and E templates
        
#     SA = tdi.noisepsd_AE(fr, model='Proposal', includewd=None)
    SA = tdi.noisepsd_AE(fr, model='Proposal', includewd=None)
    SNR2 = np.sum( np.real(DAf[ib:ie] * np.conjugate(Af.data) + DEf[ib:ie] * np.conjugate(Ef.data))/SA )
    hh = np.sum((np.absolute(Af.data)**2 + np.absolute(Ef.data)**2) /SA)
    
#     print ("SN", 4.0*df*SNR2, 4.0*df* hh)

    loglik = 4.0*df*( SNR2 - 0.5 * hh )
    return (loglik)



def LikelihoodFlat(pars):
    
    ### I presume that pars is of dimention: Nsrc x Npars
#     print ("Test", np.shape(pars))
        
    Nsrc = int(len(pars)/Npars) ## number of sources in the model 
    pars = np.reshape(pars, (Nsrc, Npars))
    
    for ind in range(Nsrc):
        pp = pars[ind, :]
        
        inPrior = True

        for i in range(len(pp)):
            if (pp[i] < prior[i,0] or pp[i]>prior[i,1]):
                inPrior=False
                # print ('didnot pass', i, pp[i], prior[i,:])

        if (not inPrior):
            return (-np.inf)
        else:
            
            # assume parameters are 
            ## log10(Amp), fr, log10(fdot), sin(beta), lambda, cos(iota), psi, phi0
            l_Amp = pp[0]
            mf0 = pp[1]
            l_fdot = pp[2]
            sin_bet = pp[3]
            lam = pp[4]
            cos_iota = pp[5]
            psi = pp[6]
            phi0 = pp[7]

            iota = np.arccos(cos_iota)
            beta = np.arcsin(sin_bet)
            Amp = 10.0**l_Amp
            f0 = mf0 
            fdot = 10.0**l_fdot

            prm = np.array([f0, fdot, beta, lam, Amp, iota, psi, -phi0])
#             print ('prm = ', prm)
            #### Constructing template
            Xf_i, Yf_i, Zf_i = fastGB.onefourier(simulator='synthlisa', params=prm, buffer=None, T=Tobs, dt=del_t, algorithm='Michele', oversample=4)
            Xf_i.df = df
            if (ind == 0):
                Xf = copy.deepcopy(Xf_i)
                Xf.df = df
                Xf.kmin = Xf_i.kmin
                Yf = copy.deepcopy(Yf_i)
                Yf.df = df
                Yf.kmin = Yf_i.kmin
                Zf = copy.deepcopy(Zf_i)
                Zf.df = df
                Zf.kmin = Zf_i.kmin
            else:
                Xf = Xf + Xf_i
                Yf = Yf + Yf_i
                Zf = Zf + Zf_i
        
#     Af = (Zf[:Nfd] - Xf[:Nfd])/np.sqrt(2.0)
#     Ef = (Zf[:Nfd] - 2.0*Yf[:Nfd] + Xf[:Nfd])/np.sqrt(6.0)
#     Af = Af/df
#     Ef = Ef/df  ### ifft normalization
    
    Af = FrequencyArray.FrequencyArray( ((Zf[:] - Xf[:])/np.sqrt(2.0))/df, dtype=np.complex128, kmin=Xf.kmin, df=Xf.df)
    Ef = FrequencyArray.FrequencyArray( ((Zf[:] - 2.0*Yf[:] + Xf[:])/np.sqrt(6.0))/df, dtype=np.complex128, kmin=Xf.kmin, df=Xf.df)

        
    ib = Xf.kmin
    ie = Xf.kmin+len(Xf) 
    fr = np.arange(Xf.kmin, Xf.kmin+len(Xf))*df
    ### TODO I assume that the frequency range is the same for A and E templates
        
#     SA = tdi.noisepsd_AE(fr, model='Proposal', includewd=None)
    SA = tdi.noisepsd_AE(fr, model='Proposal', includewd=None)
    SNR2 = np.sum( np.real(DAf[ib:ie] * np.conjugate(Af.data) + DEf[ib:ie] * np.conjugate(Ef.data))/SA )
    hh = np.sum((np.absolute(Af.data)**2 + np.absolute(Ef.data)**2) /SA)
    plotIt = False
    if plotIt:
        fig, ax = plt.subplots(nrows=2, sharex=True) 
        ax[0].plot(fr, np.abs(DAf[ib:ie]))
        ax[0].plot(fr, np.abs(Af.data))
        
        ax[1].plot(fr, np.abs(DEf[ib:ie]))
        ax[1].plot(fr, np.abs(Ef.data))
        plt.show()
        
    
#     print ("SN", 4.0*df*SNR2, 4.0*df* hh)

    loglik = 4.0*df*( SNR2 - 0.5 * hh )
    return (loglik) 


def GetFDdata(Tobs):
    
    Nt = int(Tobs/del_t)  
    XDt = td[:Nt, 1]
    YDt = td[:Nt, 2]
    ZDt = td[:Nt, 3]
    tm = td[:Nt, 0]
    win = utils.Window(tm)


    DXf = np.fft.fft(XDt)
    DYf = np.fft.fft(YDt)
    DZf = np.fft.fft(ZDt)
    freqD = np.fft.fftfreq(len(tm), del_t)

    Nfd = len(freqD)
    Nfd = int(Nfd/2)
    freqD = freqD[:Nfd]
    ### data
#     DXf = DXf[:Nfd] *del_t  

    DAf = del_t*(DZf[:Nfd] - DXf[:Nfd])/np.sqrt(2.0)
    DEf = del_t*(DXf[:Nfd] - 2.0*DYf[:Nfd] + DZf[:Nfd])/np.sqrt(6.0)
    
    return (freqD, DAf, DEf)


### Reading data and transforming to FD

In [ ]:

FD5 =  LISAhdf5("/home/stefan/LDC/Radler/data/LDC1-3_VGB_v2.hdf5")
# FD5 =  LISAhdf5("/Users/stas/Projects/LISA/ldc1_evaluation/Data/LDC1-3_VGB_v1.hdf5")
Nsrc = FD5.getSourcesNum()
GWs = FD5.getSourcesName()
print ("Found %d GW sources: " % Nsrc, GWs)
### TODO make sure GalBin is there
if (GWs[0] != 'GalBinaries'):
    raise NotImplementedError
p = FD5.getSourceParameters(GWs[0])
td = FD5.getPreProcessTDI()
del_t = float(p.get("Cadence"))
Tobs = float(p.get("ObservationDuration"))

p.display()
Nsrc = len(p.get('Frequency'))

df = 1.0/Tobs
XDt = td[:, 1]
tm = td[:, 0]
win = utils.Window(tm)


XDf = np.fft.fft(win*td[:, 1])
YDf = np.fft.fft(win*td[:, 2])
ZDf = np.fft.fft(win*td[:, 3])
freqD = np.fft.fftfreq(len(tm), del_t)
 
Nfd = len(freqD)
Nfd = int(Nfd/2)
freqD = freqD[:Nfd]  ### we use only positive frequencies
Adf = (ZDf[:Nfd] - XDf[:Nfd])/np.sqrt(2.0)
Edf = (ZDf[:Nfd] - 2.0*YDf[:Nfd] + XDf[:Nfd])/np.sqrt(6.0)
Adf = Adf * del_t
Edf = Edf * del_t  ### Due to fft conventions


fastGB = FB.FastGB("Test", dt=del_t, Tobs=Tobs, orbit="analytic")

In [ ]:
p

In [ ]:
### Dictionary of submitted results

VGBs = {'GW135962': {'BC':3, 'MM':0, 'BH':2},  
        'GW125313': {'BC':6, 'MM':9, 'BH':5},
        'GW181324': {'BC':8, 'MM':3, 'BH':1},
        'GW166667': {'BC':1, 'MM':7, 'BH':0},
        'GW194414': {'BC':5, 'MM':1, 'BH':6},
        'GW322061': {'BC':4, 'MM':6, 'BH':7},
        'GW351250': {'BC':0, 'MM':5, 'BH':9}, 
        'GW168350': {'BC':2, 'MM':4, 'BH':8},
        'GW622028': {'BC':7, 'MM':2, 'BH':4},
        'GW261301': {'BC':9, 'MM':8, 'BH':3}}

#### Read Barcelona data (from ice-csic-ieec)

In [ ]:
Barc_data = []
Barc_data_pnd = []
fls = glob.glob('/home/stefan/Repositories/ldc1_evaluation_data/submission/Barcelona/ldc1-3_posteriors_ice-csic-ieec_v2/*.csv')
print (fls)

# with open(fls[0]) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for raw in csv_reader:
#         print (raw)
#         break

### Chains are very long we thin them right away

for fl in fls:
#     df = pandas.read_csv(fl)
#     Barc_data_pnd.append(df)
    dat = np.genfromtxt(fl, delimiter=',', names=True)[::100]
    Barc_data.append(dat)


In [ ]:
print(fls)
np.shape(Barc_data)

In [ ]:
dr = '/home/stefan/Repositories/ldc1_evaluation_data/submission/ETH_LDC1-3_2/'
ETH_data = {}
for nm in VGBs.keys():
    dat = np.genfromtxt(dr+nm+".csv", delimiter=',', names=True)[::10]
    ETH_data[nm]  = dat
    print (np.shape(dat))

In [ ]:
dr = '/home/stefan/Repositories/ldc1_evaluation_data/submission/Stefan_LDC14/'
ETH_data2 = {}
for nm in VGBs.keys():
    dat = np.genfromtxt(dr+nm+".csv", delimiter=',', names=True)
    ETH_data2[nm]  = dat
    print (np.shape(dat))

In [ ]:
dr = '/home/stefan/Repositories/ldc1_evaluation_data/submission/Stefan_LDC14/GW125313_two_signals'
ETH_data3 = {}
dat = np.genfromtxt(dr+".csv", delimiter=',', names=True)
ETH_data3  = dat
print (np.shape(dat))

### Plotting ETH and Barc results (add my later) 

In [ ]:
import acor
nm = 'GW125313'

dat = ETH_data[nm]
print (acor.acor(dat['Frequency']))

In [ ]:
VGBs = {'GW135962': {'BC':3, 'MM':0, 'BH':2},  
        'GW125313': {'BC':6, 'MM':9, 'BH':5},
        'GW181324': {'BC':8, 'MM':3, 'BH':1},
        'GW166667': {'BC':1, 'MM':7, 'BH':0},
        'GW194414': {'BC':5, 'MM':1, 'BH':6},
        'GW322061': {'BC':4, 'MM':6, 'BH':7},
        'GW351250': {'BC':0, 'MM':5, 'BH':9}, 
        'GW168350': {'BC':2, 'MM':4, 'BH':8},
        'GW622028': {'BC':7, 'MM':2, 'BH':4},
        'GW261301': {'BC':9, 'MM':8, 'BH':3}}

nm = 'GW181324'

ind = VGBs[nm]['BC']
# lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\iota$', r'$A$', r'$\psi$', r'$\phi_0$']
lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\cos{\iota}$', r'$A$', r'$\psi$', r'$\phi_0$']

datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude'] + np.pi
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
# datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 4] = np.cos(Barc_data[ind]['Inclination'])
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])
datBc[:, 6] = Barc_data[ind]['Polarization']
datBc[:, 7] = Barc_data[ind]['InitialPhase']

dat = ETH_data[nm]

datETH = np.zeros((len(dat), len(lbls)))
datETH[:, 0] = dat['EclipticLongitude'] + np.pi
datETH[:, 1] = dat['EclipticLatitude']
datETH[:, 2] = dat['Frequency']
datETH[:, 3] = np.log10(np.abs(dat['FrequencyDerivative']))
# datETH[:, 4] = np.arccos(np.cos(dat['Inclination']))
datETH[:, 4] = np.cos(dat['Inclination'])
datETH[:, 5] = np.log10(dat['Amplitude'])
datETH[:, 6] = dat['Polarization']
datETH[:, 7] = dat['InitialPhase']

# dat = ETH_data2[nm]

# datETH2 = np.zeros((len(dat), len(lbls)))
# datETH2[:, 0] = dat['EclipticLongitude'] + np.pi
# datETH2[:, 1] = dat['EclipticLatitude']
# datETH2[:, 2] = dat['Frequency']
# datETH2[:, 3] = np.log10(np.abs(dat['FrequencyDerivative']))
# # datETH2[:, 4] = np.arccos(np.cos(dat['Inclination']))
# datETH2[:, 4] = np.cos(dat['Inclination'])
# datETH2[:, 5] = np.log10(dat['Amplitude'])
# datETH2[:, 6] = dat['Polarization']
# datETH2[:, 7] = dat['InitialPhase']

# dat = ETH_data3

# datETH3 = np.zeros((len(dat), len(lbls)))
# datETH3[:, 0] = dat['EclipticLongitude'] + np.pi
# datETH3[:, 1] = dat['EclipticLatitude']
# datETH3[:, 2] = dat['Frequency']
# datETH3[:, 3] = np.log10(np.abs(dat['FrequencyDerivative']))
# # datETH3[:, 4] = np.arccos(np.cos(dat['Inclination']))
# datETH3[:, 4] = np.cos(dat['Inclination'])
# datETH3[:, 5] = np.log10(dat['Amplitude'])
# datETH3[:, 6] = dat['Polarization']
# datETH3[:, 7] = dat['InitialPhase']

ind_0 = 2
bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

# tr_s = np.array([lam+np.pi, bet, f0, np.log10(fdot), np.arccos(np.cos(iota)), np.log10(Amp), psi, phi0])
tr_s = np.array([lam+np.pi, bet, f0, np.log10(fdot), np.cos(iota), np.log10(Amp), psi, phi0])
print ("true params", tr_s)



In [ ]:
print (lbls)
# rng = [0.999, 0.999, 0.999, 0.999, (0, np.pi), (-22,-21.4), (0.8, np.pi), (0, 2*np.pi)]
rng = [0.999, 0.999, 0.999, 0.999, (0, 1.1), (-22,-21.4), (0.8, np.pi), (0, 2*np.pi)]

rng = []
for i in range(len(lbls)):
    minrange = min(datETH[:,i].min(), datBc[:,i].min())
    maxrange = max(datETH[:,i].max(), datBc[:,i].max())
    oner = ( minrange, maxrange)
    rng.append(oner)


# fig1 = corner.corner(datETH,  bins=40,  hist_kwargs={'density':True, 'lw':2},  plot_datapoints=False, labels=lbls,  show_titles=True, \
#                         color='darkslategray', truths= tr_s, truth_color='k', use_math_test=True,\
#                         levels=[0.95], title_kwargs={"fontsize": 12})


fig1 = corner.corner(datETH,  bins=40,  hist_kwargs={'density':True, 'lw':2},  plot_datapoints=False, fill_contours=False,  labels=lbls,  show_titles=True, \
                        color='orangered', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.5,0.9], title_kwargs={"fontsize": 12})#, range=rng)

# corner.corner(datBc,  bins=40,  hist_kwargs={'density':True, 'lw':2}, fig=fig1, plot_datapoints=False, fill_contours=False,  labels=lbls,  show_titles=True, \
#                         color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
#                         levels=[0.9], title_kwargs={"fontsize": 12}, range=rng)

plt.show()

### How I sample the data (not a search, seeding at true position)

In [ ]:
ind_0 = 0
ks = list(VGBs.keys())
print ('this is ', ks[ind_0])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)
# print (f0, Amp, fdot)
nm_pars1 = ['log10_Amp', 'fr', 'log10_fdot', 'sin_bet', 'lam', 'cos_iota', 'psi', 'phi0'] 
Npars = 8
x0 = np.array([np.log10(Amp), f0,  np.log10(fdot), np.sin(bet), lam+2.0*np.pi, np.cos(iota), psi, phi0])


fr_my_min = f0 - 5.e-6
fr_my_max = f0 + 5.e-6
prior = MakePrior1Src(f_fctr=1.e-3)

frac = 1  ### use 1/frac part of the data
Tobs = float(p.get("ObservationDuration"))/frac
df = 1.0/Tobs

freqD, DAf, DEf = GetFDdata(Tobs)

fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(12, 8))
ax[0].plot(freqD, np.abs(DAf))
ax[0].set_xlim([fr_my_min, fr_my_max])
ax[0].set_ylim([1.e-18, 5e-17])
# ax[0].set_xlim([4.e-5, 2.e-2])

ax[0].grid(True)
ax[1].plot(freqD, np.abs(DEf))
ax[1].set_ylim([1.e-18, 5e-17])
ax[1].grid(True)

fastGB = FB.FastGB("Test", dt=del_t, Tobs=Tobs, orbit="analytic")


prm = np.array([f0, fdot, bet,  lam+2.0*np.pi,  Amp,  iota,  psi, -phi0])




#### Prepare sampler and sample

In [ ]:
np.random.seed(0)
Nchains = 3
Nadapt = 1000
arg_harm= {'maxoff':8.0}
prop_args = {'cov' : None, 'DE_skip': 1000}
out_dir = './Results/'
Npars = 8

chainLength = 500000 ###
verbal=True

x_init = [x0, x0, x0] ### seeding at true location 

Ms = mcmc.MultiChain_MCMC(np.array(prior), Nchains, LikelihoodFlat, Nadapt, nm_pars1, 1 )
Ms.InitialPoints(x_init)
Ms.InitializeProposals([{'SCAM':70., 'DE':50, 'slice':40.}, \
                        {'SCAM':70., 'DE':50., 'slice':40.},\
                        {'SCAM':70., 'DE':50., 'slice':40.}], **prop_args)



chains = Ms.runMCMC(chainLength,  verbal)

In [ ]:
chnS = np.array(chains[0].chn[100000::50])
chnS = np.concatenate((chnS, np.array(chains[1].chn[100000::50])))
chnS = np.concatenate((chnS, np.array(chains[2].chn[100000::50])))

print (np.shape(chnS))

# plt.plot(chnS[:, 1], '.')
# plt.show()

In [ ]:
nm = 'GW135962'

datSt = np.zeros((len(chnS), len(lbls)))
datSt[:,0] = chnS[:,4]-np.pi
datSt[:,1] = np.arcsin(chnS[:, 3])
datSt[:,2] = chnS[:, 1]
datSt[:,3] = chnS[:, 2]
# datSt[:,4] = np.pi-np.arccos(chSt[:, 5])
datSt[:,4] =chnS[:, 5]
datSt[:,5] = chnS[:, 0]
datSt[:, 6] = chnS[:, 6]
datSt[:, 7] =  chnS[:, 7]

ind = VGBs[nm]['BC']
# lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\iota$', r'$A$', r'$\psi$', r'$\phi_0$']
lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\cos{\iota}$', r'$A$', r'$\psi$', r'$\phi_0$']

datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude'] + np.pi
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
# datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 4] = np.cos(Barc_data[ind]['Inclination'])
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])
datBc[:, 6] = Barc_data[ind]['Polarization']
datBc[:, 7] = Barc_data[ind]['InitialPhase']

dat = ETH_data[nm]

datETH = np.zeros((len(dat), len(lbls)))
datETH[:, 0] = dat['EclipticLongitude'] + np.pi
datETH[:, 1] = dat['EclipticLatitude']
datETH[:, 2] = dat['Frequency']
datETH[:, 3] = np.log10(np.abs(dat['FrequencyDerivative']))
# datETH[:, 4] = np.arccos(np.cos(dat['Inclination']))
datETH[:, 4] = np.cos(dat['Inclination'])
datETH[:, 5] = np.log10(dat['Amplitude'])
datETH[:, 6] = dat['Polarization']
datETH[:, 7] = dat['InitialPhase']

ind_0 = 0
bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

# tr_s = np.array([lam+np.pi, bet, f0, np.log10(fdot), np.arccos(np.cos(iota)), np.log10(Amp), psi, phi0])
tr_s = np.array([lam+np.pi, bet, f0, np.log10(fdot), np.cos(iota), np.log10(Amp), psi, phi0])
print ("true params", tr_s)




In [ ]:
rng = [(0.325, 0.42), (0.25, 0.4), ( 1.35962000e-03 -3.0e-9,  1.35962000e-03 + 1.e-9), 0.999, (-1, 1), 0.999, (0., 2*np.pi), (0, 2*np.pi)]
# rng = [(0.325, 0.42), (0.25, 0.4), 0.999, (-18,-16.5), (0.1, 1.2), (-22.4, -21.6), (0, 2.0*np.pi), (0, 2.0*np.pi)]

fig2 = corner.corner(datSt,  bins=40,  hist_kwargs={'density':True, 'lw':2},  plot_datapoints=False, labels=lbls,  show_titles=True, \
                        color='darkslategray', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.95], title_kwargs={"fontsize": 12}, range = rng)


corner.corner(datBc,  bins=40,  hist_kwargs={'density':True, 'lw':2}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls,  show_titles=True, \
                        color='orangered', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12}, range=rng)

corner.corner(datETH,  bins=40,  hist_kwargs={'density':True, 'lw':2}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls,  show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12}, range=rng)

plt.show()